In [3]:
import pandas as pd

df = pd.read_csv(r"C:\Users\laptech\OneDrive\Desktop\Whale backup\train.csv")

print(df.columns)
print(df.head())



Index(['image', 'species', 'individual_id'], dtype='object')
                image             species individual_id
0  00021adfb725ed.jpg  melon_headed_whale  cadddb1636b9
1  000562241d384d.jpg      humpback_whale  1a71fbb72250
2  0007c33415ce37.jpg  false_killer_whale  60008f293a2b
3  0007d9bca26a99.jpg  bottlenose_dolphin  4b00fe572063
4  00087baf5cef7a.jpg      humpback_whale  8e5253662392


In [5]:
import pandas as pd

# Load dataset
df = pd.read_csv(r"C:\Users\laptech\OneDrive\Desktop\Whale backup\train.csv")

# Show columns
print("Columns in dataset:")
print(df.columns)

print("\nSample values from each column:\n")

for col in df.columns:
    print("Column:", col)
    print(df[col].head(5))
    print("------")



Columns in dataset:
Index(['image', 'species', 'individual_id'], dtype='object')

Sample values from each column:

Column: image
0    00021adfb725ed.jpg
1    000562241d384d.jpg
2    0007c33415ce37.jpg
3    0007d9bca26a99.jpg
4    00087baf5cef7a.jpg
Name: image, dtype: object
------
Column: species
0    melon_headed_whale
1        humpback_whale
2    false_killer_whale
3    bottlenose_dolphin
4        humpback_whale
Name: species, dtype: object
------
Column: individual_id
0    cadddb1636b9
1    1a71fbb72250
2    60008f293a2b
3    4b00fe572063
4    8e5253662392
Name: individual_id, dtype: object
------


In [6]:
print(df["species"].unique())


['melon_headed_whale' 'humpback_whale' 'false_killer_whale'
 'bottlenose_dolphin' 'beluga' 'minke_whale' 'fin_whale' 'blue_whale'
 'gray_whale' 'southern_right_whale' 'common_dolphin' 'kiler_whale'
 'pilot_whale' 'dusky_dolphin' 'killer_whale' 'long_finned_pilot_whale'
 'sei_whale' 'spinner_dolphin' 'bottlenose_dolpin' 'cuviers_beaked_whale'
 'spotted_dolphin' 'globis' 'brydes_whale' 'commersons_dolphin'
 'white_sided_dolphin' 'short_finned_pilot_whale' 'rough_toothed_dolphin'
 'pantropic_spotted_dolphin' 'pygmy_killer_whale' 'frasiers_dolphin']


In [7]:
df_whales = df[df["species"].str.contains("whale", case=False, na=False)]


In [8]:
print(df_whales["species"].unique())
print("Remaining rows:", len(df_whales))


['melon_headed_whale' 'humpback_whale' 'false_killer_whale' 'minke_whale'
 'fin_whale' 'blue_whale' 'gray_whale' 'southern_right_whale'
 'kiler_whale' 'pilot_whale' 'killer_whale' 'long_finned_pilot_whale'
 'sei_whale' 'cuviers_beaked_whale' 'brydes_whale'
 'short_finned_pilot_whale' 'pygmy_killer_whale']
Remaining rows: 26479


In [9]:
df_whales.to_csv("whales_only.csv", index=False)
print("Saved whales_only.csv")


Saved whales_only.csv


In [12]:
import os
import pandas as pd

df = pd.read_csv("whales_only.csv")

image_folder = r"C:\Users\laptech\Downloads\train_images"

missing = []

for img in df["image"]:
    if not os.path.exists(os.path.join(image_folder, img)):
        missing.append(img)

print("Missing images:", len(missing))


Missing images: 0


In [13]:
pip install torch torchvision


  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/113.8 MB ? eta -:--:--
   ---------------------------------------- 0.5/113.8 MB 2.4 MB/s eta 0:00:48
   ---------------------------------------- 1.0/113.8 MB 3.0 MB/s eta 0:00:38
   ---------------------------------------- 1.3/113.8 MB 2.0 MB/s eta 0:00:57
    --------------------------------------- 2.1/113.8 MB 2.6 MB/s eta 0:00:44
    --------------------------------------- 2.6/113.8 MB 2.7 MB/s eta 0:00:42
   - -------------------------------------- 2.9/113.8 MB 2.2 MB/s eta 0:00:50
   - -------------------------------------- 3.9/113.8 MB 2.7 MB/s eta 0:00:41
   - -------------------------------------- 4.5/113.8 MB 2.7 MB/s eta 0:00:40
   - -------------------------------------- 5.0/113.8 MB 2.7 MB/s eta 0:00:41
   -- ------------------------------------- 5.8/113.8 MB 2.8 MB/s eta 0:00:39
   -- ------------------


[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import torch
from torch.utils.data import Dataset
from PIL import Image

class WhaleDataset(Dataset):
    def __init__(self, df, image_dir, transform=None):
        self.df = df
        self.image_dir = image_dir
        self.transform = transform
        
        self.labels = df["Id"].unique()
        self.label_map = {label:i for i,label in enumerate(self.labels)}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_dir, row["Image"])
        image = Image.open(img_path).convert("RGB")
        
        label = self.label_map[row["Id"]]
        
        if self.transform:
            image = self.transform(image)
            
        return image, label


In [15]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])


In [17]:
import torchvision.models as models
import torch.nn as nn

model = models.resnet18(pretrained=True)

num_classes = len(df["individual_id"].unique())

model.fc = nn.Linear(model.fc.in_features, num_classes)


In [19]:
from torch.utils.data import Dataset
from PIL import Image
import os

class WhaleDataset(Dataset):
    def __init__(self, df, image_dir, transform=None):
        self.df = df
        self.image_dir = image_dir
        self.transform = transform

        # -------- auto detect image column --------
        self.img_col = [c for c in df.columns 
                        if "img" in c.lower() 
                        or "image" in c.lower()
                        or "file" in c.lower()][0]

        # -------- auto detect label column --------
        self.label_col = [c for c in df.columns 
                          if c.lower() not in self.img_col.lower()][0]

        print("Image column:", self.img_col)
        print("Label column:", self.label_col)

        self.labels = df[self.label_col].unique()
        self.label_map = {label:i for i,label in enumerate(self.labels)}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        img_path = os.path.join(self.image_dir, row[self.img_col])
        image = Image.open(img_path).convert("RGB")

        label = self.label_map[row[self.label_col]]

        if self.transform:
            image = self.transform(image)

        return image, label


In [20]:
dataset = WhaleDataset(df, image_folder, transform)
loader = DataLoader(dataset, batch_size=32, shuffle=True)


Image column: image
Label column: species
